In [1]:
import ast
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim

from copy import deepcopy
from sklearn.model_selection import KFold

In [2]:
#modelo MLP, 1 camada intermediára com 500 neurônios
class MLP_500_classifier(nn.Module):
    def __init__(self, input_len):
        super().__init__()

        self.layer1 = nn.Linear(input_len, 500)  
        self.relu = nn.ReLU()                          
        self.layer2 = nn.Linear(500, 30) 

    def forward(self, x):

        x = self.relu(self.layer1(x))  
        logits = self.layer2(x)
        
        return logits

In [3]:
# abrindo os dados de treinamento
df = pd.read_csv("./../ansatz_result/data.csv")
X = df.drop(columns=["target"]).to_numpy()
y = pd.DataFrame(df['target'].apply(ast.literal_eval).tolist()).to_numpy()

y_best_ansatz = np.argmax(y, axis=1)  

# criando o kfold
kf = KFold(n_splits=3, shuffle=True, random_state=42)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {device} device")

Using cpu device


In [5]:
# implementação de um critério de parada para o modelo parar no ponto "ótimo"
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = float('inf')
        self.early_stop = False
        
    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

In [6]:
# 3. Função para treinar um fold
def train_fold(model, train_loader, val_loader, criterion, optimizer, n_epochs=1000000, patience = 10, min_delta = 0.0):
    model.to(device)
    early_stopping = EarlyStopping(patience=patience, min_delta=min_delta)
    best_model = None
    best_acc = 0.0
    
    for epoch in range(n_epochs):
        # Treinamento
        model.train()
        train_loss = 0.0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_batch = y_batch.squeeze()  # Converte (batch_size, 1) para (batch_size,)
            
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # Validação
        model.eval()
        with torch.no_grad():
            val_loss = 0.0
            correct = 0
            total = 0
            for X_val, y_val in val_loader:
                X_val, y_val = X_val.to(device), y_val.to(device)
                y_val = y_val.squeeze()  # Garante que y_val é 1D
                
                outputs = model(X_val)
                val_loss += criterion(outputs, y_val).item()  
                _, predicted = torch.max(outputs.data, 1)
                correct += (predicted == y_val).sum().item()
                total += y_val.size(0)
        
        
        val_acc = correct / total
        val_loss /= len(val_loader)
        train_loss /= len(train_loader)
        
        print(f'Epoch {epoch+1}/{n_epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f} - Val Acc: {val_acc:.4f}')
        
        # Early Stopping
        early_stopping(val_loss)
        if early_stopping.early_stop:
            print(f"Early stopping at epoch {epoch+1}")
            break
            
        # Salvar melhor modelo
        if val_acc > best_acc:
            best_acc = val_acc
            best_model = deepcopy(model.state_dict())
    
    return best_model, best_acc

In [7]:
results = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"\n=== Fold {fold + 1}/{3} ===")
    
    # Dividir e preparar dados
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y_best_ansatz[train_idx], y_best_ansatz[val_idx]
    
    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.LongTensor(y_train))
    val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.LongTensor(y_val))
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32)
    
    # Inicializar modelo e otimizador
    model = MLP_500_classifier(22)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)  # L2 regularization
    
    # Treinar fold
    best_model, best_acc = train_fold(model, train_loader, val_loader, criterion, optimizer, patience= 15, min_delta=0.0)
    results.append(best_acc)
    
    # Salvar modelo se necessário
    torch.save(best_model, f'./models_salvos/best_model_MLP-500-classifier_fold{fold+1}.pt')

# 7. Resultados finais
print("\n=== Resultados ===")
print(f"Acurácia média: {np.mean(results):.4f} ± {np.std(results):.4f}")
print(f"Acurácias por fold: {results}")


=== Fold 1/3 ===
Epoch 1/1000000 - Train Loss: 2.9380 - Val Loss: 2.7584 - Val Acc: 0.2583
Epoch 2/1000000 - Train Loss: 2.3916 - Val Loss: 2.7810 - Val Acc: 0.2583
Epoch 3/1000000 - Train Loss: 2.2634 - Val Loss: 2.6666 - Val Acc: 0.2583
Epoch 4/1000000 - Train Loss: 2.2326 - Val Loss: 2.6934 - Val Acc: 0.2583
Epoch 5/1000000 - Train Loss: 2.2361 - Val Loss: 2.6914 - Val Acc: 0.2583
Epoch 6/1000000 - Train Loss: 2.1763 - Val Loss: 2.6815 - Val Acc: 0.3000
Epoch 7/1000000 - Train Loss: 2.1646 - Val Loss: 2.7638 - Val Acc: 0.2333
Epoch 8/1000000 - Train Loss: 2.1967 - Val Loss: 2.7077 - Val Acc: 0.3083
Epoch 9/1000000 - Train Loss: 2.1226 - Val Loss: 2.7104 - Val Acc: 0.3000
Epoch 10/1000000 - Train Loss: 2.1011 - Val Loss: 2.6950 - Val Acc: 0.2667
Epoch 11/1000000 - Train Loss: 2.1027 - Val Loss: 2.7882 - Val Acc: 0.3000
Epoch 12/1000000 - Train Loss: 2.0803 - Val Loss: 2.6972 - Val Acc: 0.2833
Epoch 13/1000000 - Train Loss: 2.1011 - Val Loss: 2.7530 - Val Acc: 0.2833
Epoch 14/1000000

In [8]:
model = MLP_500_classifier(22).to(device)
#carregando o melhor fold treinado
model.load_state_dict(torch.load('.//models_salvos/best_model_MLP-500-classifier_fold2.pt'))  # Substitua pelo caminho correto
model.eval()  # Modo de avaliação

MLP_500_classifier(
  (layer1): Linear(in_features=22, out_features=500, bias=True)
  (relu): ReLU()
  (layer2): Linear(in_features=500, out_features=30, bias=True)
)

In [9]:
input = X[6]
input_tensor = torch.FloatTensor(input).to(device)

with torch.no_grad():
    logits = model(input_tensor)
    print(logits)
    print(logits.shape)
    probabilities = torch.softmax(logits, dim=0)  # Shape (1, 30)

tensor([ 1.6142, -2.3243,  1.3192, -3.7483, -6.5113, -6.4406,  1.3386,  0.8577,
        -1.4171, -5.4326, -3.5050,  0.6132, -0.1423, -0.5097,  1.3137, -6.4582,
        -1.0245, -3.5299, -5.2758, -3.8636, -3.8992, -0.8746, -3.9422, -5.2814,
        -3.3296, -2.3045, -4.8350, -4.5567, -4.5225, -3.6123])
torch.Size([30])


In [10]:
print(probabilities)

tensor([2.1432e-01, 4.1746e-03, 1.5957e-01, 1.0050e-03, 6.3419e-05, 6.8064e-05,
        1.6269e-01, 1.0058e-01, 1.0342e-02, 1.8650e-04, 1.2819e-03, 7.8763e-02,
        3.7001e-02, 2.5625e-02, 1.5869e-01, 6.6879e-05, 1.5315e-02, 1.2503e-03,
        2.1816e-04, 8.9555e-04, 8.6420e-04, 1.7791e-02, 8.2784e-04, 2.1695e-04,
        1.5276e-03, 4.2578e-03, 3.3902e-04, 4.4778e-04, 4.6337e-04, 1.1514e-03])


In [11]:
# Obter índices e valores das top-k probabilidades (ex.: top-5)
k = 3
top_k_probs, top_k_indices = torch.topk(probabilities, k=k, dim=0)

# Converter para numpy e exibir
top_k_probs = top_k_probs.cpu().numpy().flatten()
top_k_indices = top_k_indices.cpu().numpy().flatten()

print("Melhores ansatzes e suas probabilidades:")
for i, (idx, prob) in enumerate(zip(top_k_indices, top_k_probs)):
    print(f"{i+1}º: Ansatz {idx} - Probabilidade: {prob:.4f}")

Melhores ansatzes e suas probabilidades:
1º: Ansatz 0 - Probabilidade: 0.2143
2º: Ansatz 6 - Probabilidade: 0.1627
3º: Ansatz 2 - Probabilidade: 0.1596
